In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset

# ============================
# 1️⃣ Data Preparation
# ============================

# Hyperparameters
SEQ_LEN = 5  # Number of previous candlesticks
FEATURES = 5  # (Open, High, Low, Close, Volume)
HIDDEN_SIZE = 64
NUM_LAYERS = 2
LEARNING_RATE = 0.001
EPOCHS = 5000
BATCH_SIZE = 32

# Simulated dataset (Replace with real data)
np.random.seed(42)
data = np.random.rand(1000, FEATURES)  # Simulating 1000 candlesticks (OHLCV)

# Normalize Data
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Convert data into sequences
X, y = [], []
for i in range(len(data) - SEQ_LEN):
    X.append(data[i:i + SEQ_LEN])
    y.append(data[i + SEQ_LEN][3])  # Predicting next Close price

X, y = np.array(X), np.array(y)

# Split Data (Train: 80%, Test: 20%)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.float32).view(-1, 1), torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ============================
# 2️⃣ LSTM Model
# ============================
class CandlestickLSTM(nn.Module):
    def __init__(self):
        super(CandlestickLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=FEATURES, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(HIDDEN_SIZE, 1)  # Predict next Close price

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Take last time step's output

# Initialize Model, Loss, Optimizer
model = CandlestickLSTM()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ============================
# 3️⃣ Training Loop + Loss Plot
# ============================
losses = []

for epoch in range(EPOCHS):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    losses.append(avg_loss)
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {avg_loss:.6f}')

# Plot Loss Curve
plt.figure(figsize=(8, 4))
plt.plot(losses, label='Training Loss', color='blue')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.show()

# ============================
# 4️⃣ Evaluation on Test Data
# ============================
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch)
        y_true.extend(y_batch.numpy().flatten())
        y_pred.extend(predictions.numpy().flatten())

# Convert Predictions Back to Original Scale
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Plot Predictions vs. Actual
plt.figure(figsize=(10, 5))
plt.plot(y_true, label="Actual Close Price", color='blue', alpha=0.7)
plt.plot(y_pred, label="Predicted Close Price", color='red', linestyle="dashed", alpha=0.8)
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.title("Predicted vs. Actual Close Price")
plt.legend()
plt.show()

# ============================
# 5️⃣ Model Performance Metrics
# ============================
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print(f"📌 Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"R² Score: {r2:.6f}")

# ============================
# 6️⃣ Prediction Function
# ============================
def predict_next_close(model, last_5_candles):
    model.eval()
    with torch.no_grad():
        input_data = torch.tensor(last_5_candles, dtype=torch.float32).unsqueeze(0)
        return model(input_data).item()

# Example Prediction
sample_input = data[-5:]  # Last 5 candlesticks
predicted_close = predict_next_close(model, sample_input)
print("Predicted Next Close Price:", predicted_close)


Epoch 0, Loss: 0.128786
Epoch 10, Loss: 0.083989
Epoch 20, Loss: 0.082453
Epoch 30, Loss: 0.082530
Epoch 40, Loss: 0.081862
Epoch 50, Loss: 0.080511
Epoch 60, Loss: 0.080161
Epoch 70, Loss: 0.079151
Epoch 80, Loss: 0.077884
Epoch 90, Loss: 0.074994
Epoch 100, Loss: 0.071784
Epoch 110, Loss: 0.069278
Epoch 120, Loss: 0.064713
Epoch 130, Loss: 0.056952
Epoch 140, Loss: 0.046053
Epoch 150, Loss: 0.037014
Epoch 160, Loss: 0.026891
Epoch 170, Loss: 0.017303
Epoch 180, Loss: 0.009546
Epoch 190, Loss: 0.005055
Epoch 200, Loss: 0.002908
Epoch 210, Loss: 0.001872
Epoch 220, Loss: 0.000728
Epoch 230, Loss: 0.000711
Epoch 240, Loss: 0.000227
Epoch 250, Loss: 0.000291
Epoch 260, Loss: 0.000582
Epoch 270, Loss: 0.000956
Epoch 280, Loss: 0.001853
Epoch 290, Loss: 0.000745
Epoch 300, Loss: 0.000364
Epoch 310, Loss: 0.000659
Epoch 320, Loss: 0.000768
Epoch 330, Loss: 0.001130
Epoch 340, Loss: 0.000419
Epoch 350, Loss: 0.000725
Epoch 360, Loss: 0.001270
Epoch 370, Loss: 0.000258
Epoch 380, Loss: 0.0003

KeyboardInterrupt: 

After adding more indicators

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset

# ============================
# 1️⃣ Technical Indicator Functions
# ============================

def compute_RSI(data, window=14):
    delta = np.diff(data)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.convolve(gain, np.ones(window)/window, mode='valid')
    avg_loss = np.convolve(loss, np.ones(window)/window, mode='valid')

    rs = avg_gain / (avg_loss + 1e-9)
    rsi = 100 - (100 / (1 + rs))
    
    return np.concatenate((np.full(window, np.nan), rsi))  # Pad with NaNs for alignment

def compute_MACD(data, short_window=12, long_window=26, signal_window=9):
    short_ema = pd.Series(data).ewm(span=short_window, adjust=False).mean()
    long_ema = pd.Series(data).ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    
    return macd.values, signal.values

def compute_Bollinger_Bands(data, window=20, num_std=2):
    rolling_mean = pd.Series(data).rolling(window=window).mean()
    rolling_std = pd.Series(data).rolling(window=window).std()

    upper_band = rolling_mean + (num_std * rolling_std)
    lower_band = rolling_mean - (num_std * rolling_std)
    
    return rolling_mean.values, upper_band.values, lower_band.values

# ============================
# 2️⃣ Data Preparation
# ============================

# Hyperparameters
SEQ_LEN = 5
FEATURES = 8  # OHLCV + RSI + MACD + Bollinger Bands
HIDDEN_SIZE = 64
NUM_LAYERS = 32
LEARNING_RATE = 0.0000001
EPOCHS = 5000
BATCH_SIZE = 32

# Simulated dataset (Replace with real data)
np.random.seed(42)
data = np.random.rand(1000, 5)  # Simulating 1000 candlesticks (OHLCV)

# Compute Indicators
close_prices = data[:, 3]  # Extract Close prices

rsi = compute_RSI(close_prices)
macd, macd_signal = compute_MACD(close_prices)
bb_middle, bb_upper, bb_lower = compute_Bollinger_Bands(close_prices)

# Align and merge features
data = np.column_stack((data, rsi, macd, bb_middle))  # Merging RSI, MACD, Bollinger Bands

# Remove NaN rows (due to indicators)
data = data[30:]  # Remove first 30 rows (max required for MACD/Bollinger Bands)

# Normalize Data
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Convert data into sequences
X, y = [], []
for i in range(len(data) - SEQ_LEN):
    X.append(data[i:i + SEQ_LEN])
    y.append(data[i + SEQ_LEN][3])  # Predict next Close price

X, y = np.array(X), np.array(y)

# Split Data (Train: 80%, Test: 20%)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.float32).view(-1, 1), torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ============================
# 3️⃣ LSTM Model
# ============================
class CandlestickLSTM(nn.Module):
    def __init__(self):
        super(CandlestickLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=FEATURES, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(HIDDEN_SIZE, 1)  # Predict next Close price

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Take last time step's output

# Initialize Model, Loss, Optimizer
model = CandlestickLSTM()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ============================
# 4️⃣ Training Loop + Loss Plot
# ============================
losses = []

for epoch in range(EPOCHS):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    losses.append(avg_loss)
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {avg_loss:.10f}')

# Plot Loss Curve
plt.figure(figsize=(8, 4))
plt.plot(losses, label='Training Loss', color='blue')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.show()

# ============================
# 5️⃣ Evaluation on Test Data
# ============================
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch)
        y_true.extend(y_batch.numpy().flatten())
        y_pred.extend(predictions.numpy().flatten())

# Convert Predictions Back to Original Scale
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Plot Predictions vs. Actual
plt.figure(figsize=(10, 5))
plt.plot(y_true, label="Actual Close Price", color='blue', alpha=0.7)
plt.plot(y_pred, label="Predicted Close Price", color='red', linestyle="dashed", alpha=0.8)
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.title("Predicted vs. Actual Close Price with Indicators")
plt.legend()
plt.show()

# ============================
# 6️⃣ Model Performance Metrics
# ============================
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print(f"📌 Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"R² Score: {r2:.6f}")

# ============================
# 7️⃣ Prediction Function
# ============================
def predict_next_close(model, last_5_candles):
    model.eval()
    with torch.no_grad():
        input_data = torch.tensor(last_5_candles, dtype=torch.float32).unsqueeze(0)
        return model(input_data).item()

# Example Prediction
sample_input = data[-5:]  # Last 5 candlesticks
predicted_close = predict_next_close(model, sample_input)
print("Predicted Next Close Price:", predicted_close)


Epoch 0, Loss: 0.3444495392
Epoch 10, Loss: 0.3470057267
Epoch 20, Loss: 0.3399769104
Epoch 30, Loss: 0.3450610167
Epoch 40, Loss: 0.3358251286
Epoch 50, Loss: 0.3404396689
Epoch 60, Loss: 0.3424739754
Epoch 70, Loss: 0.3405193228
Epoch 80, Loss: 0.3397761816
Epoch 90, Loss: 0.3412737608
Epoch 100, Loss: 0.3430178893
Epoch 110, Loss: 0.3432188410
Epoch 120, Loss: 0.3470231128


KeyboardInterrupt: 

After using CUDA

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import DataLoader, TensorDataset

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ============================
# 1️⃣ Technical Indicator Functions
# ============================
def compute_RSI(data, window=14):
    delta = np.diff(data)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.convolve(gain, np.ones(window)/window, mode='valid')
    avg_loss = np.convolve(loss, np.ones(window)/window, mode='valid')

    rs = avg_gain / (avg_loss + 1e-9)
    rsi = 100 - (100 / (1 + rs))
    
    return np.concatenate((np.full(window, np.nan), rsi))

def compute_MACD(data, short_window=12, long_window=26, signal_window=9):
    short_ema = pd.Series(data).ewm(span=short_window, adjust=False).mean()
    long_ema = pd.Series(data).ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    
    return macd.values, signal.values

def compute_Bollinger_Bands(data, window=20, num_std=2):
    rolling_mean = pd.Series(data).rolling(window=window).mean()
    rolling_std = pd.Series(data).rolling(window=window).std()
    upper_band = rolling_mean + (num_std * rolling_std)
    lower_band = rolling_mean - (num_std * rolling_std)
    
    return rolling_mean.values, upper_band.values, lower_band.values

# ============================
# 2️⃣ Data Preparation
# ============================
SEQ_LEN = 5
FEATURES = 8  # OHLCV + RSI + MACD + Bollinger Bands
HIDDEN_SIZE = 64
NUM_LAYERS = 2
LEARNING_RATE = 0.00001
EPOCHS = 50000
BATCH_SIZE = 32

# Simulated dataset (Replace with real data)
np.random.seed(42)
data = np.random.rand(1000, 5)  # Simulating 1000 candlesticks (OHLCV)

# Compute Indicators
close_prices = data[:, 3]

rsi = compute_RSI(close_prices)
macd, macd_signal = compute_MACD(close_prices)
bb_middle, bb_upper, bb_lower = compute_Bollinger_Bands(close_prices)

# Align and merge features
data = np.column_stack((data, rsi, macd, bb_middle))
data = data[30:]  # Remove first 30 rows (due to indicator calculations)

# Normalize Data
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Convert data into sequences
X, y = [], []
for i in range(len(data) - SEQ_LEN):
    X.append(data[i:i + SEQ_LEN])
    y.append(data[i + SEQ_LEN][3])  # Predict next Close price

X, y = np.array(X), np.array(y)

# Split Data (Train: 80%, Test: 20%)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Convert to PyTorch tensors & move to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ============================
# 3️⃣ LSTM Model (CUDA)
# ============================
class CandlestickLSTM(nn.Module):
    def __init__(self):
        super(CandlestickLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=FEATURES, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(HIDDEN_SIZE, 1)  # Predict next Close price

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Take last time step's output

# Initialize Model, Loss, Optimizer (Move to GPU)
model = CandlestickLSTM().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ============================
# 4️⃣ Training Loop (CUDA)
# ============================
losses = []

for epoch in range(EPOCHS):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    losses.append(avg_loss)
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {avg_loss:.10f}')

# ============================
# 5️⃣ Evaluation on Test Data (CUDA)
# ============================
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch)
        y_true.extend(y_batch.cpu().numpy().flatten())
        y_pred.extend(predictions.cpu().numpy().flatten())

# Convert Predictions Back to Original Scale
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Plot Predictions vs. Actual
plt.figure(figsize=(10, 5))
plt.plot(y_true, label="Actual Close Price", color='blue', alpha=0.7)
plt.plot(y_pred, label="Predicted Close Price", color='red', linestyle="dashed", alpha=0.8)
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.title("Predicted vs. Actual Close Price (CUDA Optimized)")
plt.legend()
plt.show()

# ============================
# 6️⃣ Model Performance Metrics
# ============================
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print(f"📌 Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"R² Score: {r2:.6f}")

# ============================
# 7️⃣ Prediction Function (CUDA)
# ============================
def predict_next_close(model, last_5_candles):
    model.eval()
    with torch.no_grad():
        input_data = torch.tensor(last_5_candles, dtype=torch.float32).unsqueeze(0).to(device)
        return model(input_data).cpu().item()

# Example Prediction
sample_input = data[-5:]  # Last 5 candlesticks
predicted_close = predict_next_close(model, sample_input)
print("Predicted Next Close Price:", predicted_close)


Using device: cuda
Epoch 0, Loss: 0.1934572201
Epoch 10, Loss: 0.1608726546
Epoch 20, Loss: 0.1213262624
Epoch 30, Loss: 0.0923125823
Epoch 40, Loss: 0.0844411919
Epoch 50, Loss: 0.0849650574
Epoch 60, Loss: 0.0842214210
Epoch 70, Loss: 0.0833795914
Epoch 80, Loss: 0.0835696551
Epoch 90, Loss: 0.0823718078
Epoch 100, Loss: 0.0842141157
Epoch 110, Loss: 0.0853668532
Epoch 120, Loss: 0.0844085881
Epoch 130, Loss: 0.0816264265
Epoch 140, Loss: 0.0821522215
Epoch 150, Loss: 0.0837482335
Epoch 160, Loss: 0.0824481919
Epoch 170, Loss: 0.0812092628
Epoch 180, Loss: 0.0810058554
Epoch 190, Loss: 0.0821929748
Epoch 200, Loss: 0.0834310687
Epoch 210, Loss: 0.0836448637
Epoch 220, Loss: 0.0824503243
Epoch 230, Loss: 0.0838371187
Epoch 240, Loss: 0.0828663249
Epoch 250, Loss: 0.0833668207
Epoch 260, Loss: 0.0832002354
Epoch 270, Loss: 0.0812972362
Epoch 280, Loss: 0.0806283715
Epoch 290, Loss: 0.0817981836
Epoch 300, Loss: 0.0823424095
Epoch 310, Loss: 0.0824819314
Epoch 320, Loss: 0.0835978228
Ep